# PSformer for Multivariate Stock Forecasting - Production Implementation

This notebook implements the **production-ready** PSformer (Parameter Shared Transformer) model for **multivariate** stock price forecasting using multiple tickers.

**Key Improvements for Production:**
- **Full Training Pipeline**: Complete training from scratch with proper train/val/test splits
- **RevIN with Lookback Window**: Better handling of non-stationary data
- **SAM Optimizer**: Sharpness-Aware Minimization for better generalization
- **MC Dropout**: Uncertainty quantification for risk management
- **Early Stopping**: Prevent overfitting with model checkpointing
- **Production Inference**: Real-time prediction pipeline

## Features:
- **Multivariate forecasting**: Predict multiple stock tickers simultaneously
- **Cross-series dependencies**: Leverage correlations between different stocks
- **Parameter sharing**: Efficient computation across all attention mechanisms
- **RevIN normalization**: Better generalization across different price scales
- **Two-stage segment attention**: Enhanced feature extraction
- **Risk quantification**: Prediction intervals for uncertainty assessment

## Expected Data Format:
```
Date,AAPL_Close,GOOGL_Close,MSFT_Close,TSLA_Close
2023-01-01,150.0,100.0,250.0,200.0
2023-01-02,152.0,101.5,252.3,205.1
...
```

In [ ]:
# Install required packages
!pip install torch pandas numpy matplotlib plotly scikit-learn seaborn tqdm

# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from typing import Tuple, Optional, Dict, Any, List
from tqdm import tqdm
import warnings
import os
import json
from datetime import datetime
warnings.filterwarnings('ignore')

print("Setup complete! PyTorch version:", torch.__version__)

# Configuration

This cell contains all the parameters for production multivariate time series forecasting.

In [ ]:
# ========== DATA CONFIGURATION ==========
DATA_FILE_PATH = "stock_data.csv"  # Change this to your uploaded CSV file name
DATE_COLUMN = "Date"
# MULTIVARIATE APPROACH: Each ticker becomes a variable (column)
# Expected format: Date, AAPL_Close, GOOGL_Close, MSFT_Close, ...
TICKER_SYMBOLS = ['VCB_Close', 'VIC_Close', 'VHM_Close', 'BID_Close', 'TCB_Close', 'CTG_Close', 'HPG_Close', 'VPB_Close', 'FPT_Close', 'MBB_Close']  # Update with your actual ticker columns

# ========== MODEL HYPERPARAMETERS ==========
SEQUENCE_LENGTH = 96    # Input sequence length (L)
PATCH_SIZE = 16          # Temporal patch size (P)
PREDICTION_LENGTH = 30   # Forecast horizon (F)
NUM_ENCODER_LAYERS = 1   # Number of PSformer encoder layers (optimal for financial data per paper)
NUM_VARIABLES = len(TICKER_SYMBOLS)  # Number of stock tickers in multivariate setting
D_MODEL = 256           # Model dimension (from paper)
N_HEADS = 8             # Number of attention heads
REVIN_LOOKBACK_WINDOW = 16  # RevIN lookback window for non-stationary data

# ========== TRAINING CONFIGURATION ==========
BATCH_SIZE = 32
MAX_EPOCHS = 100
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 1e-4
PATIENCE = 10           # Early stopping patience
TRAIN_SPLIT = 0.7       # 70% for training
VAL_SPLIT = 0.15        # 15% for validation
TEST_SPLIT = 0.15       # 15% for testing

# ========== SAM OPTIMIZER CONFIGURATION ==========
USE_SAM = True          # Use Sharpness-Aware Minimization
SAM_RHO = 0.15          # SAM hyperparameter for perturbation radius (optimal 0.1-0.2 for financial data)

# ========== UNCERTAINTY QUANTIFICATION ==========
MC_DROPOUT_SAMPLES = 100  # Number of samples for Monte Carlo Dropout
DROPOUT_RATE = 0.1      # Dropout rate for uncertainty

# ========== VALIDATION CONFIGURATION ==========
MIN_DATA_POINTS = SEQUENCE_LENGTH + PREDICTION_LENGTH  # Minimum required data

# ========== DEVICE CONFIGURATION ==========
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Verify configuration
print(f"\nProduction Configuration Summary:")
print(f"- Input sequence length: {SEQUENCE_LENGTH} days")
print(f"- Patch size: {PATCH_SIZE} days")
print(f"- Number of patches: {SEQUENCE_LENGTH // PATCH_SIZE}")
print(f"- Prediction horizon: {PREDICTION_LENGTH} days")
print(f"- RevIN lookback window: {REVIN_LOOKBACK_WINDOW} days")
print(f"- Stock tickers: {NUM_VARIABLES} ({', '.join(TICKER_SYMBOLS)})")
print(f"- Training mode: {'SAM Optimizer' if USE_SAM else 'Standard Adam'}")
print(f"- Uncertainty quantification: MC Dropout with {MC_DROPOUT_SAMPLES} samples")
print(f"- Architecture: Two-stage segment attention with ReLU (per paper Figure 2)")

# Production PSformer Implementation

The following cells contain the complete source code for the production-ready PSformer model with all enhancements.

In [ ]:
# ========== PRODUCTION REVIN IMPLEMENTATION ==========
class RevIN(nn.Module):
    """Production RevIN with lookback window for better non-stationary data handling"""
    
    def __init__(self, num_features: int, eps=1e-5, affine=True, lookback_window: Optional[int] = None):
        """
        :param num_features: the number of features or channels
        :param eps: a value added for numerical stability
        :param affine: if True, RevIN has learnable affine parameters
        :param lookback_window: if specified, use only last N time steps for statistics
        """
        super(RevIN, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        self.lookback_window = lookback_window
        if self.affine:
            self._init_params()

    def forward(self, x, mode: str):
        if mode == 'norm':
            self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        else: 
            raise NotImplementedError
        return x

    def _init_params(self):
        # initialize RevIN params: (C,)
        self.affine_weight = nn.Parameter(torch.ones(self.num_features))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

    def _get_statistics(self, x):
        """Calculate statistics with optional lookback window for better non-stationary handling"""
        if self.lookback_window is not None:
            # Use only the last 'lookback_window' time steps for statistics
            x_stats = x[:, :, -self.lookback_window:]
        else:
            x_stats = x
        
        # Calculate statistics across the time dimension (last dimension)
        self.mean = torch.mean(x_stats, dim=-1, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x_stats, dim=-1, keepdim=True, unbiased=False) + self.eps).detach()

    def _normalize(self, x):
        x = x - self.mean
        x = x / self.stdev
        if self.affine:
            # Reshape for proper broadcasting: [C] -> [1, C, 1]
            weight = self.affine_weight.view(1, -1, 1)
            bias = self.affine_bias.view(1, -1, 1)
            x = x * weight
            x = x + bias
        return x

    def _denormalize(self, x):
        if self.affine:
            # Reshape for proper broadcasting: [C] -> [1, C, 1]
            weight = self.affine_weight.view(1, -1, 1)
            bias = self.affine_bias.view(1, -1, 1)
            x = x - bias
            x = x / (weight + self.eps*self.eps)
        x = x * self.stdev
        x = x + self.mean
        return x

In [ ]:
# ========== PS BLOCK IMPLEMENTATION ==========
class PSBlock(nn.Module):
    """
    Parameter Shared Block implementing Equation 3 from PSformer paper:
    Xout = (GeLU(XinW(1))W(2) + Xin)W(3)
    """
    
    def __init__(self, N: int):
        """
        Args:
            N: Dimension size for N×N weight matrices
        """
        super().__init__()
        self.N = N
        
        # Three N×N linear layers with bias
        self.linear1 = nn.Linear(N, N)
        self.linear2 = nn.Linear(N, N) 
        self.linear3 = nn.Linear(N, N)
        
        # Activation function
        self.activation = nn.GELU()
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        """Initialize weights using Xavier initialization for W1, W2 and smaller weights for W3"""
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)
        # Initialize linear3 with smaller weights as it's the final transformation
        nn.init.xavier_uniform_(self.linear3.weight, gain=0.1)
        
        if self.linear1.bias is not None:
            nn.init.zeros_(self.linear1.bias)
        if self.linear2.bias is not None:
            nn.init.zeros_(self.linear2.bias)
        if self.linear3.bias is not None:
            nn.init.zeros_(self.linear3.bias)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass implementing the three-step transformation
        
        Args:
            x: Input tensor of shape (C, N) or (batch, C, N)
            
        Returns:
            Output tensor of same shape as input
        """
        # Handle both 2D and 3D tensors
        original_shape = x.shape
        is_3d = x.dim() == 3
        
        # Validate input shape
        if x.dim() not in [2, 3]:
            raise ValueError(f"Input tensor must be 2 or 3-dimensional, got {x.dim()}")
        
        if is_3d:
            # Reshape 3D to 2D: [batch, C, N] -> [batch*C, N]
            batch, C, N = x.shape
            if N != self.N:
                raise ValueError(f"Input tensor last dimension must be {self.N}, got {N}")
            x = x.view(-1, N)  # [batch*C, N]
        else:
            # 2D case
            if x.shape[1] != self.N:
                raise ValueError(f"Input tensor second dimension must be {self.N}, got {x.shape[1]}")
        
        # Store original input for residual connection
        residual = x
        
        # First transformation: Linear -> GeLU -> Linear + Residual
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        intermediate_output = x + residual
        
        # Second transformation: Linear
        final_output = self.linear3(intermediate_output)
        
        # Reshape back to original shape if needed
        if is_3d:
            final_output = final_output.view(batch, C, N)
        
        return final_output

In [ ]:
# ========== ATTENTION MECHANISM WITH DROPOUT ==========
class ScaledDotProductAttention(nn.Module):
    """Scaled Dot-Product Attention with dropout for uncertainty quantification"""
    
    def __init__(self, dropout_rate: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.scale = None  # Will be computed dynamically based on input
    
    def forward(self, Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """Compute scaled dot-product attention with dropout"""
        # Validate input dimensions
        if Q.dim() != 3 or K.dim() != 3 or V.dim() != 3:
            raise ValueError("Q, K, and V must all be 3-dimensional tensors")
        
        if Q.shape[0] != K.shape[0] or Q.shape[0] != V.shape[0]:
            raise ValueError("Batch dimensions of Q, K, and V must match")
            
        if K.shape[1] != V.shape[1]:
            raise ValueError("Key and Value must have the same sequence length")
            
        if Q.shape[2] != K.shape[2]:
            raise ValueError("Query and Key must have the same feature dimension")
        
        # Compute scaling factor
        dk = Q.shape[2]
        self.scale = 1.0 / torch.sqrt(torch.tensor(dk, dtype=torch.float32, device=Q.device))
        
        # Compute attention scores: Q @ K^T
        scores = torch.matmul(Q, K.transpose(-2, -1))  # [batch, num_queries, num_keys]
        
        # Apply scaling
        scaled_scores = scores * self.scale
        
        # Apply softmax to get attention weights
        attention_weights = F.softmax(scaled_scores, dim=-1)  # [batch, num_queries, num_keys]
        
        # Apply dropout to attention weights
        attention_weights = self.dropout(attention_weights)
        
        # Apply attention weights to values
        output = torch.matmul(attention_weights, V)  # [batch, num_queries, dv]
        
        return output, attention_weights


class PSformerEncoderLayer(nn.Module):
    """Single PSformer encoder layer with two-stage segment attention as per paper Figure 2"""
    
    def __init__(self, ps_block: PSBlock, dropout_rate: float = 0.1):
        super().__init__()
        # Maximum parameter sharing: same PSBlock used for QKV generation and final transformation
        self.ps_block = ps_block
        
        # Two separate attention mechanisms for the two stages
        self.attention_stage1 = ScaledDotProductAttention(dropout_rate=dropout_rate)
        self.attention_stage2 = ScaledDotProductAttention(dropout_rate=dropout_rate)
        
        # ReLU activation between the two attention stages
        self.activation = nn.ReLU()
        
        # Additional dropout for residual connections
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:
        """
        Forward pass implementing two-stage attention as per paper Figure 2:
        x -> PSBlock -> Stage1 Attention -> ReLU -> Stage2 Attention -> + x -> PSBlock
        """
        # Store input for residual connection
        residual = x
        
        # Step 1: Apply PS Block to generate Q, K, V for first stage
        qkv = self.ps_block(x)  # [batch, C, N]
        
        # Step 2: Stage 1 Attention
        stage1_output, stage1_weights = self.attention_stage1(qkv, qkv, qkv)
        
        # Step 3: ReLU activation (critical non-linearity between stages)
        stage1_activated = self.activation(stage1_output)
        
        # Step 4: Stage 2 Attention (using activated output from stage 1)
        stage2_output, stage2_weights = self.attention_stage2(stage1_activated, stage1_activated, stage1_activated)
        
        # Step 5: Residual connection with dropout
        output_with_residual = residual + self.dropout(stage2_output)
        
        # Step 6: Final PS Block transformation (as per paper architecture)
        final_output = self.ps_block(output_with_residual)
        
        # Return both attention weight tensors for analysis
        attention_weights = {
            'stage1': stage1_weights,
            'stage2': stage2_weights
        }
        
        return final_output, attention_weights


class PSformerEncoder(nn.Module):
    """Complete PSformer encoder with multiple layers"""
    
    def __init__(self, num_layers: int, segment_length: int, dropout_rate: float = 0.1):
        super().__init__()
        # Each layer has its own PS Block
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            ps_block = PSBlock(N=segment_length)
            encoder_layer = PSformerEncoderLayer(ps_block, dropout_rate=dropout_rate)
            self.layers.append(encoder_layer)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, List[Dict[str, torch.Tensor]]]:
        """Forward pass through the PSformer encoder"""
        if x.dim() != 3:
            raise ValueError(f"Input tensor must be 3-dimensional, got {x.dim()}")
        
        attention_weights_list = []
        
        # Process through each layer
        for layer in self.layers:
            x, weights = layer(x)
            attention_weights_list.append(weights)
        
        return x, attention_weights_list

In [ ]:
# ========== PRODUCTION PSFORMER MODEL ==========
class PSformerConfig:
    """Configuration class for PSformer model parameters"""
    def __init__(self, 
                 sequence_length: int,
                 num_variables: int, 
                 patch_size: int,
                 num_encoder_layers: int,
                 prediction_length: int,
                 d_model: int = 256,
                 n_heads: int = 8,
                 affine_revin: bool = True,
                 revin_eps: float = 1e-5,
                 revin_lookback_window: Optional[int] = None,
                 dropout_rate: float = 0.1):
        self.sequence_length = sequence_length
        self.num_variables = num_variables
        self.patch_size = patch_size
        self.num_encoder_layers = num_encoder_layers
        self.prediction_length = prediction_length
        self.d_model = d_model
        self.n_heads = n_heads
        self.affine_revin = affine_revin
        self.revin_eps = revin_eps
        self.revin_lookback_window = revin_lookback_window
        self.dropout_rate = dropout_rate
        
        # Validate configuration
        self._validate()
    
    def _validate(self):
        """Validate configuration parameters"""
        if self.sequence_length % self.patch_size != 0:
            raise ValueError(f"Sequence length {self.sequence_length} must be divisible by patch size {self.patch_size}")
        if self.num_variables <= 0:
            raise ValueError(f"Number of variables must be positive, got {self.num_variables}")
        if self.patch_size <= 0:
            raise ValueError(f"Patch size must be positive, got {self.patch_size}")


class PSformer(nn.Module):
    """Production PSformer model with enhanced features"""
    
    def __init__(self, config: PSformerConfig):
        super().__init__()
        self.config = config
        
        # Calculate derived parameters
        self.num_patches = config.sequence_length // config.patch_size
        self.segment_length = config.num_variables * config.patch_size  # C = M * P
        
        # RevIN normalization with lookback window
        self.revin = RevIN(
            config.num_variables, 
            eps=config.revin_eps, 
            affine=config.affine_revin,
            lookback_window=config.revin_lookback_window
        )
        
        # PSformer encoder with dropout
        self.encoder = PSformerEncoder(
            config.num_encoder_layers, 
            self.segment_length,
            dropout_rate=config.dropout_rate
        )
        
        # Output projection layer with dropout
        self.output_projection = nn.Sequential(
            nn.Dropout(config.dropout_rate),
            nn.Linear(self.segment_length, config.prediction_length)
        )
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        """Initialize model weights"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass through the PSformer model"""
        # Validate input shape
        if x.dim() != 3:
            raise ValueError(f"Input tensor must be 3-dimensional, got {x.dim()}")
        
        batch_size, num_variables, sequence_length = x.shape
        
        if num_variables != self.config.num_variables:
            raise ValueError(f"Expected {self.config.num_variables} variables, got {num_variables}")
        
        if sequence_length != self.config.sequence_length:
            raise ValueError(f"Expected sequence length {self.config.sequence_length}, got {sequence_length}")
        
        # Step 1: RevIN normalization
        x_norm = self.revin(x, 'norm')  # [batch, num_variables, sequence_length]
        
        # Step 2: Patching - reshape to segments
        x_patches = x_norm.view(batch_size, num_variables, self.num_patches, self.config.patch_size)
        x_segments = x_patches.permute(0, 2, 1, 3).contiguous()
        x_segments = x_segments.view(batch_size, self.num_patches, self.segment_length)
        
        # Step 3: PSformer encoder
        encoded_output, attention_weights = self.encoder(x_segments)
        
        # Step 4: Output projection
        predictions = self.output_projection(encoded_output)
        
        # Aggregate predictions from all patches (simple mean)
        aggregated_predictions = torch.mean(predictions, dim=1, keepdim=True)
        
        # Expand to match number of variables
        output = aggregated_predictions.expand(batch_size, num_variables, self.config.prediction_length)
        
        # Step 5: RevIN denormalization
        output = self.revin(output, 'denorm')
        
        return output

In [ ]:
# ========== SAM OPTIMIZER IMPLEMENTATION ==========
class SAM(torch.optim.Optimizer):
    """Sharpness-Aware Minimization optimizer for better generalization"""
    
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"
        
        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super(SAM, self).__init__(params, defaults)
        
        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups
    
    @torch.no_grad()
    def first_step(self, zero_grad=False):
        """First step: compute and apply perturbation"""
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)
            
            for p in group["params"]:
                if p.grad is None: continue
                e_w = (torch.pow(p, 2) if group["adaptive"] else 1.0) * p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w
        
        if zero_grad: self.zero_grad()
    
    @torch.no_grad()
    def second_step(self, zero_grad=False):
        """Second step: apply actual parameter update"""
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"
        
        self.base_optimizer.step()  # do the actual "sharpness-aware" update
        
        if zero_grad: self.zero_grad()
    
    @torch.no_grad()
    def step(self, closure=None):
        """Combined step function"""
        assert closure is not None, "SAM requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass
        
        self.first_step(zero_grad=True)
        closure()
        self.second_step()
    
    def _grad_norm(self):
        """Compute gradient norm"""
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        
        # Collect gradients
        grads = [
            ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad).norm(dtype=torch.float32)
            for group in self.param_groups for p in group["params"]
            if p.grad is not None
        ]
        
        # Handle case when no gradients exist
        if len(grads) == 0:
            return torch.tensor(0.0, device=shared_device)
        
        norm = torch.norm(torch.stack(grads), dim=0).to(shared_device)
        return norm
    
    def load_state_dict(self, state_dict):
        super().load_state_dict(state_dict)
        self.base_optimizer.param_groups = self.param_groups

In [ ]:
# ========== PRODUCTION DATA PIPELINE ==========
class StockDataset(Dataset):
    """Production dataset for sliding window time series data"""
    
    def __init__(self, dataframe: pd.DataFrame, ticker_symbols: List[str], 
                 sequence_length: int, prediction_length: int):
        """
        Args:
            dataframe: DataFrame with ticker price columns
            ticker_symbols: List of ticker column names
            sequence_length: Input sequence length
            prediction_length: Target sequence length
        """
        self.data = dataframe[ticker_symbols].values.astype(np.float32)
        self.sequence_length = sequence_length
        self.prediction_length = prediction_length
        
        # Validate data
        if len(self.data) < sequence_length + prediction_length:
            raise ValueError(f"Not enough data points. Need at least {sequence_length + prediction_length}, got {len(self.data)}")
    
    def __len__(self):
        """Total number of possible sequences"""
        return len(self.data) - self.sequence_length - self.prediction_length + 1
    
    def __getitem__(self, index):
        """Get a single sequence pair"""
        input_start = index
        input_end = index + self.sequence_length
        target_end = input_end + self.prediction_length
        
        input_seq = self.data[input_start:input_end]  # [seq_len, num_vars]
        target_seq = self.data[input_end:target_end]   # [pred_len, num_vars]
        
        # Transpose to [num_vars, seq_len] format expected by model
        input_tensor = torch.from_numpy(input_seq.T)
        target_tensor = torch.from_numpy(target_seq.T)
        
        return input_tensor, target_tensor


def split_data(df: pd.DataFrame, train_ratio: float = 0.7, val_ratio: float = 0.15) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Split data chronologically into train/val/test sets"""
    n_total = len(df)
    train_end = int(n_total * train_ratio)
    val_end = train_end + int(n_total * val_ratio)
    
    train_df = df.iloc[:train_end].copy()
    val_df = df.iloc[train_end:val_end].copy()
    test_df = df.iloc[val_end:].copy()
    
    print(f"Data split: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")
    
    return train_df, val_df, test_df


def create_data_loaders(train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame,
                       ticker_symbols: List[str], sequence_length: int, prediction_length: int,
                       batch_size: int = 32) -> Tuple[DataLoader, DataLoader, DataLoader]:
    """Create PyTorch data loaders for training"""
    
    train_dataset = StockDataset(train_df, ticker_symbols, sequence_length, prediction_length)
    val_dataset = StockDataset(val_df, ticker_symbols, sequence_length, prediction_length)
    test_dataset = StockDataset(test_df, ticker_symbols, sequence_length, prediction_length)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    print(f"Data loaders created: Train={len(train_loader)} batches, Val={len(val_loader)} batches, Test={len(test_loader)} batches")
    
    return train_loader, val_loader, test_loader

In [ ]:
# ========== UNCERTAINTY QUANTIFICATION WITH MC DROPOUT ==========
def mc_dropout_predict(model: PSformer, input_tensor: torch.Tensor, 
                      num_samples: int = 100) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Perform Monte Carlo Dropout for uncertainty quantification
    
    Args:
        model: Trained PSformer model
        input_tensor: Input tensor [batch, num_vars, seq_len]
        num_samples: Number of MC samples
        
    Returns:
        mean_prediction: Mean prediction across samples
        lower_bound: 5th percentile (lower confidence bound)
        upper_bound: 95th percentile (upper confidence bound)
    """
    model.train()  # Keep dropout active!
    
    all_predictions = []
    
    with torch.no_grad():
        for _ in range(num_samples):
            prediction = model(input_tensor)
            all_predictions.append(prediction)
    
    # Stack predictions: [num_samples, batch, vars, pred_len]
    predictions_tensor = torch.stack(all_predictions)
    
    # Calculate statistics across the sample dimension
    mean_prediction = torch.mean(predictions_tensor, dim=0)
    lower_bound = torch.quantile(predictions_tensor, 0.05, dim=0)
    upper_bound = torch.quantile(predictions_tensor, 0.95, dim=0)
    
    return mean_prediction, lower_bound, upper_bound


def plot_predictions_with_uncertainty(dates: List[str], actual: np.ndarray, 
                                    predicted: np.ndarray, lower: np.ndarray, upper: np.ndarray,
                                    ticker_name: str):
    """Plot predictions with uncertainty bands"""
    fig = go.Figure()
    
    # Actual values
    fig.add_trace(go.Scatter(
        x=dates, y=actual,
        mode='lines', name='Actual',
        line=dict(color='blue', width=2)
    ))
    
    # Predicted mean
    fig.add_trace(go.Scatter(
        x=dates, y=predicted,
        mode='lines', name='Predicted (Mean)',
        line=dict(color='red', width=2)
    ))
    
    # Confidence interval
    fig.add_trace(go.Scatter(
        x=dates + dates[::-1],
        y=upper.tolist() + lower[::-1].tolist(),
        fill='toself', fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip", showlegend=True,
        name='90% Confidence Interval'
    ))
    
    fig.update_layout(
        title=f'{ticker_name} - Prediction with Uncertainty',
        xaxis_title='Date',
        yaxis_title='Price',
        hovermode='x unified'
    )
    
    fig.show()

In [ ]:
# ========== PRODUCTION TRAINING PIPELINE ==========
def train_model(model: PSformer, train_loader: DataLoader, val_loader: DataLoader,
               num_epochs: int = 100, learning_rate: float = 0.0001, 
               patience: int = 10, use_sam: bool = True, sam_rho: float = 0.05,
               device: torch.device = torch.device('cpu')) -> Dict[str, Any]:
    """
    Production training loop with early stopping and model checkpointing
    
    Returns:
        Dictionary containing training history and best model state
    """
    model = model.to(device)
    criterion = nn.MSELoss()
    
    # Initialize optimizer
    if use_sam:
        base_optimizer = torch.optim.AdamW
        optimizer = SAM(model.parameters(), base_optimizer, rho=sam_rho, lr=learning_rate, weight_decay=1e-4)
        print(f"Using SAM optimizer with rho={sam_rho}")
    else:
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
        print("Using standard AdamW optimizer")
    
    # Training tracking
    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []
    best_model_state = None
    
    print(f"Starting training for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        # ========== TRAINING STEP ==========
        model.train()
        total_train_loss = 0.0
        num_train_batches = 0
        
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        
        for batch_inputs, batch_targets in train_pbar:
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            
            if use_sam:
                # SAM requires a closure for the second forward pass
                def closure():
                    optimizer.zero_grad()
                    predictions = model(batch_inputs)
                    loss = criterion(predictions, batch_targets)
                    loss.backward()
                    return loss
                
                loss = optimizer.step(closure)
            else:
                # Standard training step
                optimizer.zero_grad()
                predictions = model(batch_inputs)
                loss = criterion(predictions, batch_targets)
                loss.backward()
                optimizer.step()
            
            total_train_loss += loss.item()
            num_train_batches += 1
            
            # Update progress bar
            train_pbar.set_postfix({'Loss': f'{loss.item():.6f}'})
        
        avg_train_loss = total_train_loss / num_train_batches
        train_losses.append(avg_train_loss)
        
        # ========== VALIDATION STEP ==========
        model.eval()
        total_val_loss = 0.0
        num_val_batches = 0
        
        with torch.no_grad():
            for batch_inputs, batch_targets in val_loader:
                batch_inputs = batch_inputs.to(device)
                batch_targets = batch_targets.to(device)
                
                predictions = model(batch_inputs)
                loss = criterion(predictions, batch_targets)
                
                total_val_loss += loss.item()
                num_val_batches += 1
        
        avg_val_loss = total_val_loss / num_val_batches
        val_losses.append(avg_val_loss)
        
        # ========== EARLY STOPPING & MODEL SAVING ==========
        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.6f}, Val Loss = {avg_val_loss:.6f}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
            print(f"✓ New best model saved (Val Loss: {best_val_loss:.6f})")
        else:
            patience_counter += 1
            print(f"No improvement ({patience_counter}/{patience})")
            
            if patience_counter >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs!")
                break
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Loaded best model with validation loss: {best_val_loss:.6f}")
    
    return {
        'model': model,
        'train_losses': train_losses,
        'val_losses': val_losses,
        'best_val_loss': best_val_loss,
        'best_model_state': best_model_state
    }


def save_model_checkpoint(model: PSformer, model_state: dict, config: PSformerConfig, 
                         training_history: dict, filepath: str):
    """Save complete model checkpoint for production deployment"""
    checkpoint = {
        'model_state_dict': model_state,
        'config': {
            'sequence_length': config.sequence_length,
            'num_variables': config.num_variables,
            'patch_size': config.patch_size,
            'num_encoder_layers': config.num_encoder_layers,
            'prediction_length': config.prediction_length,
            'd_model': config.d_model,
            'n_heads': config.n_heads,
            'affine_revin': config.affine_revin,
            'revin_eps': config.revin_eps,
            'revin_lookback_window': config.revin_lookback_window,
            'dropout_rate': config.dropout_rate
        },
        'training_history': training_history,
        'timestamp': datetime.now().isoformat()
    }
    
    torch.save(checkpoint, filepath)
    print(f"Model checkpoint saved to: {filepath}")


def load_model_checkpoint(filepath: str, device: torch.device) -> Tuple[PSformer, dict]:
    """Load complete model checkpoint for production inference"""
    checkpoint = torch.load(filepath, map_location=device)
    
    # Reconstruct config
    config_dict = checkpoint['config']
    config = PSformerConfig(**config_dict)
    
    # Reconstruct model
    model = PSformer(config)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"Model checkpoint loaded from: {filepath}")
    print(f"Training timestamp: {checkpoint.get('timestamp', 'Unknown')}")
    
    return model, checkpoint

# Data Loading and Preprocessing

In [ ]:
# Load and validate data
print(f"Loading data from: {DATA_FILE_PATH}")

try:
    df = pd.read_csv(DATA_FILE_PATH)
    print(f"✓ Data loaded successfully: {len(df)} rows")
    print(f"Columns: {list(df.columns)}")
    
    # Validate required columns
    missing_columns = [col for col in TICKER_SYMBOLS if col not in df.columns]
    if missing_columns:
        print(f"❌ Missing columns: {missing_columns}")
        print("Please update TICKER_SYMBOLS or check your data file.")
    else:
        print(f"✓ All required ticker columns found: {TICKER_SYMBOLS}")
        
        # Check data sufficiency
        if len(df) < MIN_DATA_POINTS:
            print(f"❌ Insufficient data: {len(df)} rows, need at least {MIN_DATA_POINTS}")
        else:
            print(f"✓ Sufficient data: {len(df)} rows (minimum: {MIN_DATA_POINTS})")
            
            # Display data info
            print("\nData Info:")
            print(df[TICKER_SYMBOLS].describe())
            
            # Check for missing values
            missing_values = df[TICKER_SYMBOLS].isnull().sum()
            if missing_values.any():
                print(f"\n⚠️ Missing values detected:")
                print(missing_values[missing_values > 0])
                
                # Forward fill missing values
                df[TICKER_SYMBOLS] = df[TICKER_SYMBOLS].fillna(method='ffill').fillna(method='bfill')
                print("✓ Missing values handled with forward/backward fill")
            else:
                print("✓ No missing values detected")
    
except FileNotFoundError:
    print(f"❌ File not found: {DATA_FILE_PATH}")
    print("Please upload your data file or update the DATA_FILE_PATH variable.")
except Exception as e:
    print(f"❌ Error loading data: {str(e)}")

In [ ]:
# Split data chronologically and create data loaders
if 'df' in locals() and len(df) >= MIN_DATA_POINTS:
    print("Splitting data chronologically...")
    
    # Sort by date if date column exists
    if DATE_COLUMN in df.columns:
        df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
        df = df.sort_values(DATE_COLUMN).reset_index(drop=True)
        print(f"✓ Data sorted by {DATE_COLUMN}")
        print(f"Date range: {df[DATE_COLUMN].min()} to {df[DATE_COLUMN].max()}")
    
    # Split data
    train_df, val_df, test_df = split_data(df, TRAIN_SPLIT, VAL_SPLIT)
    
    # Create data loaders
    try:
        train_loader, val_loader, test_loader = create_data_loaders(
            train_df, val_df, test_df,
            TICKER_SYMBOLS, SEQUENCE_LENGTH, PREDICTION_LENGTH,
            BATCH_SIZE
        )
        print("✓ Data loaders created successfully")
        
        # Test a batch
        sample_batch = next(iter(train_loader))
        inputs, targets = sample_batch
        print(f"✓ Sample batch shape: inputs={inputs.shape}, targets={targets.shape}")
        
    except Exception as e:
        print(f"❌ Error creating data loaders: {str(e)}")
        
else:
    print("❌ Cannot proceed without valid data")

# Model Training

In [ ]:
# Initialize model and start training
if 'train_loader' in locals():
    print("Initializing PSformer model for production training...")
    
    # Create model configuration
    config = PSformerConfig(
        sequence_length=SEQUENCE_LENGTH,
        num_variables=NUM_VARIABLES,
        patch_size=PATCH_SIZE,
        num_encoder_layers=NUM_ENCODER_LAYERS,
        prediction_length=PREDICTION_LENGTH,
        d_model=D_MODEL,
        n_heads=N_HEADS,
        revin_lookback_window=REVIN_LOOKBACK_WINDOW,
        dropout_rate=DROPOUT_RATE
    )
    
    # Initialize model
    model = PSformer(config)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"✓ Model initialized")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    
    # Start training
    print(f"\nStarting production training...")
    print(f"Configuration: {MAX_EPOCHS} epochs, batch size {BATCH_SIZE}, patience {PATIENCE}")
    print(f"Optimizer: {'SAM' if USE_SAM else 'AdamW'} with LR {LEARNING_RATE}")
    
    training_results = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=MAX_EPOCHS,
        learning_rate=LEARNING_RATE,
        patience=PATIENCE,
        use_sam=USE_SAM,
        sam_rho=SAM_RHO,
        device=DEVICE
    )
    
    # Save trained model
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = f"psformer_multivariate_production_{timestamp}.pth"
    
    save_model_checkpoint(
        model=training_results['model'],
        model_state=training_results['best_model_state'],
        config=config,
        training_history={
            'train_losses': training_results['train_losses'],
            'val_losses': training_results['val_losses'],
            'best_val_loss': training_results['best_val_loss']
        },
        filepath=model_path
    )
    
    print(f"\n🎉 Training completed! Best validation loss: {training_results['best_val_loss']:.6f}")
    
else:
    print("❌ Cannot start training without data loaders")

In [ ]:
# Plot training history
if 'training_results' in locals():
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    epochs = range(1, len(training_results['train_losses']) + 1)
    
    ax.plot(epochs, training_results['train_losses'], 'b-', label='Training Loss', linewidth=2)
    ax.plot(epochs, training_results['val_losses'], 'r-', label='Validation Loss', linewidth=2)
    
    # Mark best epoch
    best_epoch = np.argmin(training_results['val_losses']) + 1
    best_val_loss = training_results['best_val_loss']
    ax.axvline(x=best_epoch, color='green', linestyle='--', alpha=0.7, label=f'Best Model (Epoch {best_epoch})')
    ax.scatter([best_epoch], [best_val_loss], color='green', s=100, zorder=5)
    
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss (MSE)')
    ax.set_title('PSformer Training History')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add text box with final stats
    final_train_loss = training_results['train_losses'][-1]
    textstr = f'Final Train Loss: {final_train_loss:.6f}\nBest Val Loss: {best_val_loss:.6f}\nBest Epoch: {best_epoch}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', bbox=props)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Training completed in {len(training_results['train_losses'])} epochs")
    print(f"Final training loss: {final_train_loss:.6f}")
    print(f"Best validation loss: {best_val_loss:.6f} (Epoch {best_epoch})")

# Model Evaluation with Uncertainty Quantification

In [ ]:
# Evaluate model on test set with uncertainty quantification
if 'training_results' in locals() and 'test_loader' in locals():
    print("Evaluating model on test set with uncertainty quantification...")
    
    model = training_results['model']
    model.eval()
    
    # Standard evaluation
    all_predictions = []
    all_targets = []
    
    model.eval()  # Standard evaluation mode
    with torch.no_grad():
        for batch_inputs, batch_targets in test_loader:
            batch_inputs = batch_inputs.to(DEVICE)
            batch_targets = batch_targets.to(DEVICE)
            
            predictions = model(batch_inputs)
            
            all_predictions.append(predictions.cpu())
            all_targets.append(batch_targets.cpu())
    
    # Concatenate all predictions and targets
    all_predictions = torch.cat(all_predictions, dim=0)  # [total_samples, num_vars, pred_len]
    all_targets = torch.cat(all_targets, dim=0)
    
    # Calculate metrics
    mse = F.mse_loss(all_predictions, all_targets).item()
    mae = F.l1_loss(all_predictions, all_targets).item()
    
    print(f"\nTest Set Performance:")
    print(f"MSE: {mse:.6f}")
    print(f"MAE: {mae:.6f}")
    print(f"RMSE: {np.sqrt(mse):.6f}")
    
    # Uncertainty quantification on a sample
    print(f"\nPerforming uncertainty quantification on sample batch...")
    sample_batch = next(iter(test_loader))
    sample_inputs, sample_targets = sample_batch
    sample_inputs = sample_inputs.to(DEVICE)
    
    # MC Dropout prediction
    mean_pred, lower_bound, upper_bound = mc_dropout_predict(
        model, sample_inputs, num_samples=MC_DROPOUT_SAMPLES
    )
    
    # Convert to numpy for plotting
    mean_pred_np = mean_pred[0].cpu().numpy()  # First sample
    lower_bound_np = lower_bound[0].cpu().numpy()
    upper_bound_np = upper_bound[0].cpu().numpy()
    actual_np = sample_targets[0].cpu().numpy()
    
    print(f"✓ Uncertainty quantification completed")
    print(f"Prediction confidence interval width (average): {np.mean(upper_bound_np - lower_bound_np):.4f}")
    
    # Plot uncertainty for first ticker
    ticker_idx = 0
    ticker_name = TICKER_SYMBOLS[ticker_idx]
    
    dates = [f"Day {i+1}" for i in range(PREDICTION_LENGTH)]
    
    plot_predictions_with_uncertainty(
        dates=dates,
        actual=actual_np[ticker_idx],
        predicted=mean_pred_np[ticker_idx],
        lower=lower_bound_np[ticker_idx],
        upper=upper_bound_np[ticker_idx],
        ticker_name=ticker_name
    )
    
else:
    print("❌ Cannot evaluate without trained model and test data")

# Production Inference Pipeline

In [ ]:
# Production inference function
def production_inference(model_path: str, data_df: pd.DataFrame, ticker_symbols: List[str],
                        use_uncertainty: bool = True, num_mc_samples: int = 100) -> Dict[str, Any]:
    """
    Production inference pipeline for real-time predictions
    
    Args:
        model_path: Path to saved model checkpoint
        data_df: DataFrame with latest data
        ticker_symbols: List of ticker column names
        use_uncertainty: Whether to compute uncertainty with MC Dropout
        num_mc_samples: Number of MC samples for uncertainty
        
    Returns:
        Dictionary with predictions and metadata
    """
    print(f"Loading model from: {model_path}")
    
    # Load model
    model, checkpoint = load_model_checkpoint(model_path, DEVICE)
    config = PSformerConfig(**checkpoint['config'])
    
    # Validate input data
    if len(data_df) < config.sequence_length:
        raise ValueError(f"Need at least {config.sequence_length} data points, got {len(data_df)}")
    
    # Prepare input tensor (last sequence_length points)
    latest_data = data_df[ticker_symbols].tail(config.sequence_length).values.astype(np.float32)
    input_tensor = torch.from_numpy(latest_data.T).unsqueeze(0).to(DEVICE)  # [1, num_vars, seq_len]
    
    print(f"Input tensor shape: {input_tensor.shape}")
    
    if use_uncertainty:
        print(f"Computing predictions with uncertainty ({num_mc_samples} samples)...")
        mean_pred, lower_bound, upper_bound = mc_dropout_predict(model, input_tensor, num_mc_samples)
        
        # Convert to numpy
        predictions = mean_pred[0].cpu().numpy()  # [num_vars, pred_len]
        lower = lower_bound[0].cpu().numpy()
        upper = upper_bound[0].cpu().numpy()
        
        return {
            'predictions': predictions,
            'lower_bound': lower,
            'upper_bound': upper,
            'ticker_symbols': ticker_symbols,
            'prediction_length': config.prediction_length,
            'uncertainty_quantified': True,
            'num_mc_samples': num_mc_samples,
            'model_info': {
                'training_timestamp': checkpoint.get('timestamp', 'Unknown'),
                'best_val_loss': checkpoint['training_history']['best_val_loss']
            }
        }
    else:
        print("Computing standard predictions...")
        model.eval()
        with torch.no_grad():
            predictions = model(input_tensor)
        
        predictions_np = predictions[0].cpu().numpy()  # [num_vars, pred_len]
        
        return {
            'predictions': predictions_np,
            'ticker_symbols': ticker_symbols,
            'prediction_length': config.prediction_length,
            'uncertainty_quantified': False,
            'model_info': {
                'training_timestamp': checkpoint.get('timestamp', 'Unknown'),
                'best_val_loss': checkpoint['training_history']['best_val_loss']
            }
        }


# Example production inference
if 'model_path' in locals() and 'df' in locals():
    print("\n" + "="*50)
    print("PRODUCTION INFERENCE EXAMPLE")
    print("="*50)
    
    try:
        # Run production inference
        results = production_inference(
            model_path=model_path,
            data_df=df,
            ticker_symbols=TICKER_SYMBOLS,
            use_uncertainty=True,
            num_mc_samples=50  # Reduced for faster demo
        )
        
        # Display results
        print(f"\n🎯 Production Predictions Generated!")
        print(f"Model trained: {results['model_info']['training_timestamp']}")
        print(f"Model performance: {results['model_info']['best_val_loss']:.6f} (validation loss)")
        print(f"Prediction horizon: {results['prediction_length']} days")
        print(f"Uncertainty quantified: {results['uncertainty_quantified']}")
        
        # Show sample predictions
        print(f"\nSample Predictions (first 5 days):")
        for i, ticker in enumerate(results['ticker_symbols'][:3]):  # Show first 3 tickers
            pred = results['predictions'][i][:5]  # First 5 days
            if results['uncertainty_quantified']:
                lower = results['lower_bound'][i][:5]
                upper = results['upper_bound'][i][:5]
                print(f"{ticker}: {pred} [CI: {lower} - {upper}]")
            else:
                print(f"{ticker}: {pred}")
        
        print(f"\n✅ Production inference completed successfully!")
        print(f"💡 Use this pipeline to make real-time predictions on new data.")
        
    except Exception as e:
        print(f"❌ Production inference failed: {str(e)}")
        
else:
    print("❌ Cannot run production inference without trained model")

# Summary

## Production Features Implemented:

✅ **RevIN with Lookback Window**: Enhanced normalization for non-stationary data  
✅ **Complete Training Pipeline**: Train/val/test splits with early stopping  
✅ **SAM Optimizer**: Sharpness-Aware Minimization for better generalization  
✅ **Monte Carlo Dropout**: Uncertainty quantification for risk management  
✅ **Model Checkpointing**: Save/load complete model states  
✅ **Production Inference**: Real-time prediction pipeline  

## Next Steps for Production Deployment:

1. **Hyperparameter Tuning**: Use grid search or Bayesian optimization
2. **Data Pipeline**: Implement real-time data ingestion
3. **Model Monitoring**: Track prediction accuracy over time
4. **A/B Testing**: Compare different model versions
5. **API Wrapper**: Create REST API for inference
6. **Containerization**: Docker deployment for scalability

## Risk Management Features:

- **Uncertainty Bounds**: 90% confidence intervals for all predictions
- **Model Validation**: Proper train/val/test splits prevent overfitting
- **Early Stopping**: Prevents model degradation
- **Robust Training**: SAM optimizer finds flatter minima

This implementation is now **production-ready** with proper training, validation, and uncertainty quantification capabilities.